In [1]:
import sys
import os
sys.path.insert(0, "../../python")
sys.path.insert(0, "../../litos")

import cProfile
%load_ext line_profiler
from line_profiler import LineProfiler
%load_ext memory_profiler

import numpy as np
import particle_beam as pb
import plasma_source as ps
import particle_beam_propagation as pbp
import scipy.constants as const

In [3]:
# define plasma bulk (flat-top) properties
npl0   = 1e16 # cm^-3
dEds0  = np.sqrt(npl0/(1e17))*10e9 # eV/m
dgds0  = 0.0#dEds0/const.m_e
L_ft   = 0.50 # m

# define plasma up-ramp
shape_up = 'gauss'
hw_up    = 0.1205 #0.05 # m
L_up     = 1.5 # m
top_up   = L_up # m

# define plasma down-ramp
shape_dn = shape_up
hw_dn    = hw_up # m
L_dn     = L_up # m
top_dn   = 0  # m

# define beam parameters
gbC    = 20000 # relativistic lorentz factor
eps    = 5.0e-6  # m-rad, normalized emittance
beta   = 0.10  # m

alpha  = 0.00
gamma  = (1.0+alpha**2)/beta # 1/m
dgb    = 0.01
dz     = 0
npart  = 10000
dist   = 'gauss'

# make beam
s0     = 0.0
twiss = pb.make_twiss(beta,alpha,gamma,eps,gbC,dgb,dz)
parts = pb.make_parts(twiss[0],npart,dist)
ebeam = pb.make_ebeam(s0,twiss[0],parts[0])

# add offset to bunch at vac. waist
ebeam[0]["x"] += 10e-6 # m

# set beam waist position
waist = -0.28 #-0.105 # m, waist location w.r.t L_up
s_w   = L_up + waist # m

# define longitudinal steps
wp0    = (5.64e4)*np.sqrt(npl0) # rad/s, plasma ang. freq.
kp0    = wp0/const.c # m^-1, plasma wave number
kb     = kp0/np.sqrt(2*gbC)

ds   = (1.0/kb)*(1./20.) # m
print(ds)
s_ft = np.linspace(0,L_ft,int(L_ft/ds+1))
s_up = np.linspace(0,L_up,int(L_up/ds+1))
s_dn = np.linspace(0,L_dn,int(L_dn/ds+1))

# make plasma
bulk    = ps.make_bulk(s_ft,npl0,dgds0)
up_ramp = ps.make_ramp(s_up,'up',shape_up,hw_up,top_up,npl0,dgds0)
dn_ramp = ps.make_ramp(s_dn,'dn',shape_dn,hw_dn,top_dn,npl0,dgds0)
plasma  = ps.make_plasma(bulk,up_ramp,dn_ramp)

# propagate beam backward from waist to start of plasma
pbp.prop_ebeam_drift(ebeam,[0,-s_w],last_only=True)
twiss = pb.get_twiss(ebeam,len(ebeam)-1)
parts = pb.get_parts(ebeam,len(ebeam)-1)
ebeam = pb.make_ebeam(s0,twiss[len(ebeam)-1],parts[len(ebeam)-1])
vbeam = ebeam.copy()

# propagate beam through plasma
cProfile.run('pbp.prop_ebeam_plasma(ebeam,plasma,last_only=False)')

# propagate beam through vacuum
pbp.prop_ebeam_drift(vbeam,plasma["s"],last_only=False)

0.000531546911348
         320247 function calls in 414.087 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:402(parent)
       10    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.001    0.001  414.087  414.087 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 _collections_abc.py:657(get)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:664(__contains__)
        1    0.000    0.000    0.000    0.000 _parallel_backends.py:120(effective_n_jobs)
        1    0.000    0.000    0.060    0.060 _parallel_backends.py:132(terminate)
        8    0.000    0.000    0.000    0.000 _parallel_backends.py:139(apply_async)
        9    0.000    0.000    0.000    0.000 _parallel_backends.py:168(compute_batch_size)
        1    0.000    0.000    0.000    0.000 _parallel_backends.py